## Simulation of Full version of Kuang's model

### Using package

In [1]:
using Pkg;
Pkg.activate("/work/b11209013/external/JuliaENV/atmo/");

using Plots, Measures; gr();
using FFTW, LinearAlgebra, Statistics;
using LazyGrids;

BLAS.set_num_threads(1)

default(
    size=(1600,900),
    dpi=100,
    bottom_margin=10mm,
    left_margin=10mm,
    top_margin=10mm,
    right_margin=10mm,
    titlefont=font(32,"times"),
    guidefont=font(28,"times"),
    tickfont=font(24,"times"),
    legendfont=font(18,"times")
)

  Activating project at `/work/b11209013/external/JuliaENV/atmo`


### Functions

#### System Matrix

In [2]:
function sys_matrix(
    k :: Float64;
    ϵ=ϵ, c1=c1, c2=c2,
    a1=a1, a2=a2, d1=d1, d2=d2, rq=rq, r0=r0,
    A=A, B=B, f=f, τL=τL
    )

    α :: ComplexF64 = 1.5*rq*(d1+d2);
    β :: ComplexF64 = -rq*(d1-d2);
    γ :: ComplexF64 = -1*((d1-d2)*r0 + (d1+d2));

    mat :: Matrix{ComplexF64} = [
    -ϵ 0.0 (c1*k)^2 0.0 0.0 0.0;
    0.0 -ϵ 0.0 (c2*k)^2 0.0 0.0;
    -1.0 0.0 -1.5*rq 0.0 rq 1+r0;
    0.0 -1.0 1.5*rq 0.0 -rq 1-r0;
    a1 a2 α 0.0 β γ;
    f/B/τL (1-f)/B/τL -1.5*A*rq/B/τL 0.0 A*rq/B/τL -1.0/τL
                                ];
    return mat
end

sys_matrix (generic function with 1 method)

### Set constant

In [3]:
const b1 :: Float64 = 1.0;
const b2 :: Float64 = 2.0;

const a1 :: Float64 = 1.4;
const a2 :: Float64 = 0.0;

const d1 :: Float64 = 1.1;
const d2 :: Float64 = -1.0;

const m1 :: Float64 = 0.3;
const m2 :: Float64 = 1.0;

const r0 :: Float64 = 1.0;
const rq :: Float64 = 0.7;

const F  :: Float64 = 4.0;
const f  :: Float64 = 0.5;

const τL :: Float64 = 1/12;
const c1 :: Float64 = 1.0;
const c2 :: Float64 = 0.5;

const ϵ  :: Float64 = 0.1;

const Γ  :: Float64 = 6.5/1000.0;

const Rd :: Float64 = 287.5;
const Cp :: Float64 = 1004.5;
const g  :: Float64 = 9.81;

const A  :: Float64 = 1.0 - 2.0*f + (b2-b1)/F;
const B  :: Float64 = 1.0 + (b2+b1)/F - A*r0;

### Configuring domain and wavenumber

In [4]:
# spatial domain
## Horizontal
Δx       :: Float64         = 1e3; # spatial interval: 1 km;
Lx       :: Float64         = 4e7; # Maximum length: 40000 km;
x        :: Vector{Float64} = collect(-Lx:Δx:Lx);
x_nondim :: Vector{Float64} = x ./ 4320000;
Nx       :: Int64           = length(x); # length of x-domain (80001,);

## Vertical
Δz :: Float64         = 1e2;  # Vertical spacing: 100 m;
Lz :: Float64         = 15e3; # Depth of troposphere;
z  :: Vector{Float64} = collect(0.0:Δz:Lz);
Nz :: Int64           = length(z);

## Vertical modes
# G_j(z) = π/2 sin(jπz/H_T), where j=1,2
G1 :: Matrix{Float64} = reshape(π/2*sin.(π*z/Lz), Nz, 1); # first vertical mode
G2 :: Matrix{Float64} = reshape(π/2*sin.(2π*z/Lz), Nz, 1);

# temporal domain
Δt :: Float64         = 1e-1; # Time interval: 0.1 day;
Lt :: Float64         = 1e2; # Final time stamp: 100 days;
t  :: Vector{Float64} = collect(0.0:Δt:Lt);
Nt :: Int64           = length(t);

# wavenumber setting
λ  :: Vector{Float64} = collect(540:540:40000);
kn :: Vector{Float64} = 2*π*4320.0./λ;

### Setting Initial condition

In [5]:
# Initial field in frequency domain
initial_state_vec :: Matrix{ComplexF64} = randn(6, length(kn))*0.1;
## initial_state_vec: row1: w1; row2: w2; row3: T1; row4: T2; row5: q; row6: L;

# inverse matrix
exp_mat :: Matrix{ComplexF64} = zeros(length(kn), Nx);

for (i, kj) in enumerate(kn)
    exp_mat[i,:] = @. exp(im*kj*x_nondim)
end ;

### Simulation


In [6]:
# Compute in frequency domain
state_vec :: Array{ComplexF64,3} = zeros(Nt, 6, length(kn));

for (j, kj) in enumerate(kn)
    mat :: Matrix{ComplexF64} = sys_matrix(kj);

    for (i, ti) in enumerate(t)
        state_vec[i,:,j] = exp(mat.*ti) * reshape(initial_state_vec[:,j],6,1);
    end
end

# choose specific Wavenumber
idx_8640 :: Int64 = argmin(abs.(kn .- 2*π*4320.0/8640.0));

final_choose :: Array{ComplexF64,3} = zeros(size(state_vec));
final_choose[:,:,idx_8640] = state_vec[:,:,idx_8640];

# reconstruct
recon :: Array{Float64,3} = zeros(Nt,6,Nx);

for i in 1:Nt
    recon[i,:,:] = real.(final_choose[i,:,:]*exp_mat);
end

### Plot out animation

In [ ]:
lft_bnd :: Int64 = argmin(abs.(x.+4320000));
rgt_bnd :: Int64 = argmin(abs.(x.-4320000));

temp_prof :: Array{Float64,3} = zeros(Nt, Nz, length(x[lft_bnd:rgt_bnd]));

for i in 1:Nt
    tot_prof = G1*reshape(recon[i,3,:],1,Nx) .+ G2*reshape(recon[i,4,:],1,Nx)

    temp_prof[i,:,:] = tot_prof[:,lft_bnd:rgt_bnd]
end

vmin, vmax = extrema(temp_prof)

anim = @animate for (t, A) in enumerate(eachslice(temp_prof; dims=1))  # A is Nz×Nx at time t
    heatmap(x[lft_bnd:rgt_bnd], z, A;                 # transpose so x runs horizontally
        xlabel = "x", ylabel = "z",
        title  = "t = $t / $Nt",
        clims  = (vmin, vmax),
        colorbar = true, framestyle = :box)
end

gif(anim, "/home/b11209013/2025_Research/MSI/Fig/temp_prof.gif"; fps = 50)